In [9]:
# Natural language processing library.
import csv, nltk, pickle, time
import pandas as pd
import numpy as np
from io import StringIO
from collections import Counter
from scipy import sparse
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

# need to use once to download nltk on your computer.
# nltk.download()

Vu qu'on a une librarie qui nous permet de faire pas mal de choses, on va:
- mettre tout en minuscule
- retirer la ponctuation
- retirer tous les nombres et caractères non alphanumériques
- les tokeniser: donc séparer les mots
- retirer les stop words (and, the, etc)
- stemmer -> avoir que la racine de chaque mots

à partir de là on aura déjà un dataset plus ou moins propre =)

In [2]:
# This function does the job of extracting tweet data and formatting it to suit our needs
def format_tweets(filename):
    # Read the data file
    with open("twitter-datasets/" + filename, "r", encoding="utf8") as myfile:
        data = myfile.readlines()
        
    # Make a dataframe out of the data
    tweets = pd.DataFrame(data)
    
    # These are the words we want to ignore
    lang_set = nltk.corpus.stopwords.words('english')

    # The replacement instructions for below
    replacements = [
        ("<user>", ''),
        ("<url>", ''),
        (r'[^\w\s]', ''),
        (r'[\d]', '')
    ]
        
    # Put everything in lowercase
    tweets[0] = tweets[0].astype(str).str.lower()
    
    # Take out :
    # - usertags
    # - urls
    # - punctuation and non-alphanumerical characters
    # - numbers
    for key, value in replacements:
        tweets[0] = tweets[0].str.replace(key, value)
        
    # Tokenize each tweet
    tweets[0] = tweets[0].str.split()
    
    # Remove the stop words and the "rt"s (retweets)
    for item in [lang_set, 'rt']:
        tweets[0] = tweets[0].apply(lambda tweet: [word for word in tweet if word not in item])
        
    return tweets

In [3]:
negative_DF = format_tweets("train_neg.txt")
positive_DF = format_tweets("train_pos.txt")
test_DF = format_tweets("test_data.txt")

In [4]:
negative_DF.head(10)

,0
0,"[vinco, tresorpack, difficulty, object, disass..."
1,"[glad, dot, taks, tomorrow, thankful, startho]"
2,"[vs, celtics, regular, season, fucked, play, p..."
3,"[could, actually, kill, girl, im, sorry]"
4,"[find, hard, believe, im, afraid]"
5,"[wish, could, night, tonight]"
6,"[got, kicked, wgm]"
7,"[yes, u, tell, lips, closed, okay]"
8,[perfect]
9,"[hi, harry, u, havea, good, time, aus, didnt, ..."


J'ai remarké que stemmetize n'est pas toujours la meilleure chose à faire et tente du coup aussi de lemmetize. Ca nous rajoute de ce faite deux colonnes, pour qu'on puisse ensuite sélectionner la meilleure à utiliser.

In [5]:
def stem_and_lem(tweets, stemmer, lemmer):
    tweets['stemmed'] = tweets[0].apply(lambda tweet: [stemmer.stem(word) for word in tweet])
    tweets['lemmed'] = tweets[0].apply(lambda tweet: [lemmatizer.lemmatize(word) for word in tweet])
    tweets['both'] = tweets['lemmed'].apply(lambda tweet: [stemmer.stem(word) for word in tweet])
    
    return tweets

In [6]:
# Generating the stemmer and lemmatizer
stemmer = nltk.stem.snowball.SnowballStemmer('english') 
lemmatizer = nltk.stem.WordNetLemmatizer()

Les lemmetizer ne semble pas amrcher à 100% donc essayons de faire les deux (donc utiliser le stem sur le lemmetizer)

In [7]:
negative_DF = stem_and_lem(negative_DF, stemmer, lemmatizer)
positive_DF = stem_and_lem(positive_DF, stemmer, lemmatizer)
test_DF = stem_and_lem(test_DF, stemmer, lemmatizer)

In [8]:
negative_DF.head(10)

,0,stemmed,lemmed,both
0,"[vinco, tresorpack, difficulty, object, disass...","[vinco, tresorpack, difficulti, object, disass...","[vinco, tresorpack, difficulty, object, disass...","[vinco, tresorpack, difficulti, object, disass..."
1,"[glad, dot, taks, tomorrow, thankful, startho]","[glad, dot, tak, tomorrow, thank, startho]","[glad, dot, taks, tomorrow, thankful, startho]","[glad, dot, tak, tomorrow, thank, startho]"
2,"[vs, celtics, regular, season, fucked, play, p...","[vs, celtic, regular, season, fuck, play, play...","[v, celtic, regular, season, fucked, play, pla...","[v, celtic, regular, season, fuck, play, playoff]"
3,"[could, actually, kill, girl, im, sorry]","[could, actual, kill, girl, im, sorri]","[could, actually, kill, girl, im, sorry]","[could, actual, kill, girl, im, sorri]"
4,"[find, hard, believe, im, afraid]","[find, hard, believ, im, afraid]","[find, hard, believe, im, afraid]","[find, hard, believ, im, afraid]"
5,"[wish, could, night, tonight]","[wish, could, night, tonight]","[wish, could, night, tonight]","[wish, could, night, tonight]"
6,"[got, kicked, wgm]","[got, kick, wgm]","[got, kicked, wgm]","[got, kick, wgm]"
7,"[yes, u, tell, lips, closed, okay]","[yes, u, tell, lip, close, okay]","[yes, u, tell, lip, closed, okay]","[yes, u, tell, lip, close, okay]"
8,[perfect],[perfect],[perfect],[perfect]
9,"[hi, harry, u, havea, good, time, aus, didnt, ...","[hi, harri, u, havea, good, time, aus, didnt, ...","[hi, harry, u, havea, good, time, au, didnt, g...","[hi, harri, u, havea, good, time, au, didnt, g..."


In [9]:
# This function saves the processed tweets to a txt file
def save_tweets(tweets, filename):
    # Put the stemmed and lemmetized tweets back to string form
    data = tweets['both'].apply(lambda x: ' '.join(x))
    
    # Save to file
    with open("twitter-datasets/" + filename, "w", encoding="utf8") as myfile:
        data.to_csv(myfile, index=False)

In [10]:
save_tweets(negative_DF, "train_neg_proc.txt")
save_tweets(positive_DF, "train_pos_proc.txt")
save_tweets(test_DF, "test_data_proc.txt")

Maintenant, il va falloir faire ce qui est proposé dans le pdf, donc par exemple, compter les mots qui apparaissent le plus dans negatif et positif, et pourquoi pas utiliser ça ensuite dans notre algorithme pour décider si c'est positif ou négatif dans le test. =)

Quelques petites notes: 
- chaque tweet est un liste de mots
- on a quatre colonnes: les mots de base du tweet, les mots mais stemmés (racine du mot), les mots mais lemmés (idem mais se veut plus précis) et un qui fait les deux (lem puis stem)
- je pars du principe que tout est en anglais.

In [11]:
# Get embeddings and index values
emb = np.load("outputs/embeddings.npy")

vocab_cut = pd.read_csv("outputs/vocab_cut.txt", sep=" ", header=None, quoting=csv.QUOTE_NONE)
index = pd.Series(vocab_cut[vocab_cut.columns[0]].values)

# Create word definition matrix
word_weights = pd.DataFrame(data=emb, index=index)

word_weights.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
im,0.093155,-0.195444,0.034837,0.503743,0.017677,0.070631,0.056530,-0.204557,-0.413193,0.165088,0.039284,-0.288781,0.237835,-0.006965,-0.162107,0.108911,-0.091243,0.234972,0.076061,0.112683
go,0.132050,-0.252235,0.033528,0.627656,0.004004,0.085370,0.067080,-0.252409,-0.525088,0.265527,0.048909,-0.352476,0.310018,-0.007982,-0.220616,0.131504,-0.134969,0.303717,0.078098,0.141692
love,0.128605,-0.273047,0.027388,0.693572,0.033754,0.070030,0.095573,-0.282355,-0.575062,0.268360,0.043465,-0.384105,0.334852,0.014419,-0.255275,0.140315,-0.136424,0.339160,0.089009,0.159651
get,0.143687,-0.299162,0.061555,0.735509,-0.000145,0.091735,0.091049,-0.305811,-0.568576,0.298758,0.044261,-0.399258,0.343303,0.003744,-0.272364,0.162020,-0.171673,0.360334,0.108146,0.155568
follow,0.277644,-0.475451,0.086347,1.071386,-0.000148,0.146430,0.067929,-0.428727,-0.847339,0.390039,0.088062,-0.580147,0.535491,-0.005055,-0.417348,0.247671,-0.190140,0.539185,0.167775,0.211712


In [12]:
# This function removes words that are not in the vocabulary from the tweets
def clean_tweets(tweets, vocab):
    clean = tweets.copy().apply(lambda tweet: [word for word in tweet if word in vocab.values])
    return clean

<h2>FastText</h2>

In [13]:
# Extract just the column we need
neg_DF = negative_DF["both"]
pos_DF = positive_DF["both"]
test_DF = test_DF["both"]

In [14]:
pos_DF.head()

0    [dunno, justin, read, mention, justin, god, kn...
1    [logic, dumb, wont, even, crop, name, photo, tsk]
2          [put, casper, box, loov, battl, crakkbitch]
3    [thank, sir, dont, trip, lil, mama, keep, doin...
4    [visit, brother, tmr, bestest, birthday, gift,...
Name: both, dtype: object

In [15]:
# We want a sparse matrix that tells us for each tweet (rows) how many times a given
# word (columns) appears ("bags of words" representation)

# MIGHT NEED TO BE REDEFINED WITH neg_words AND pos_words INSTEAD !!!!!!
def bags_of_words(tweets=None):
    
    # We get the vocabulary
    with open('outputs/vocab.pkl', 'rb') as f:
        vocab = pickle.load(f)
        
    col = 0
    data, rows, cols = [], [], []
    
    for tweet in tweets:
        # We count word occurences in a tweet
        word_count = Counter(tweet).items()
        
        # If the word is in the vocabulary we add it in the matrix
        for word, count in word_count:
            row = vocab.get(word)
            
            if row:
                data.append(count)
                cols.append(col)
                rows.append(row)
        
        col += 1
        
    # We convert the scipy.sparse matrix to pandas.SparseSeries for ease of use
    return pd.SparseSeries.from_coo(sparse.coo_matrix((data, (rows, cols))))

In [16]:
bags = bags_of_words(pos_DF)

In [17]:
bags.head()

1  12    1
   20    1
   26    1
   56    1
   89    1
dtype: int64
BlockIndex
Block locations: array([0], dtype=int32)
Block lengths: array([5], dtype=int32)

<h2>SVM</h2>

In [41]:
# Sums the embeddings of each word in the given tweet
def query_weights(tweet):
    w = pd.DataFrame(columns=range(20))
    
    for word in tweet:
        try:
            w = w.append(word_weights.loc[word, :])
        except KeyError:
            pass
        
    return w.sum(axis=0)

In [19]:
# Build tweet embeddings
#neg_dims = neg_DF.copy().apply(query_weights)
#pos_dims = pos_DF.copy().apply(query_weights)
test_dims = test_DF.copy().apply(query_weights)

KeyboardInterrupt: 

In [ ]:
# Save the embeddings in pkl files
with open('outputs/neg_dims.pkl', 'wb') as f:
    pickle.dump(neg_dims, f, pickle.HIGHEST_PROTOCOL)
    
with open('outputs/pos_dims.pkl', 'wb') as f:
    pickle.dump(pos_dims, f, pickle.HIGHEST_PROTOCOL)

In [3]:
# Load the embeddings from pkl files
with open('outputs/neg_dims.pkl', 'rb') as f:
    neg_dims = pickle.load(f)
    
with open('outputs/pos_dims.pkl', 'rb') as f:
    pos_dims = pickle.load(f)

In [4]:
# Generate the matrices for SVM fitting, we just put the positive and negative embeddings together and
# create the appropriate y matrix with 1's and -1's
X = pos_dims.append(neg_dims)
ones = np.ones((pos_dims.shape[0], 1))
y = np.append(ones, -1 * ones)

In [10]:
start = time.time()
clf = RandomForestClassifier(min_samples_leaf=20)
clf.fit(X, y)
end = time.time()
print("Random Forest", end - start, clf.score(X,y))
proba = clf.predict_proba(X)

Random Forest 17.576520204544067 0.789465
